In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# Define dataset path
data_dir = "./Data"

# Image parameters
img_height, img_width = 128, 128  # Increased resolution for better accuracy
batch_size = 32

# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

# Training data
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Compute class weights (if classes are imbalanced)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))

# Load MobileNetV2 as base model
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base layers

# Build the final model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_data.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping & learning rate adjustment
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Train the model
epochs = 50
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    class_weight=class_weights,
    callbacks=[early_stop, lr_scheduler]
)

# Save the trained model
model.save("sign_language_model_transfer_alpha.keras")

# Evaluate the model
test_loss, test_acc = model.evaluate(val_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


Found 62959 images belonging to 36 classes.
Found 15739 images belonging to 36 classes.


C:\Users\Kalik\hand_env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
1968/1968 ━━━━━━━━━━━━━━━━━━━━ 1840s 931ms/step - accuracy: 0.2695 - loss: 2.6761 - val_accuracy: 0.4904 - val_loss: 1.7418 - learning_rate: 0.0010
Epoch 2/50
1968/1968 ━━━━━━━━━━━━━━━━━━━━ 1030s 524ms/step - accuracy: 0.4908 - loss: 1.6531 - val_accuracy: 0.5378 - val_loss: 1.4640 - learning_rate: 0.0010
Epoch 3/50
1968/1968 ━━━━━━━━━━━━━━━━━━━━ 744s 378ms/step - accuracy: 0.5272 - loss: 1.4664 - val_accuracy: 0.5779 - val_loss: 1.3483 - learning_rate: 0.0010
Epoch 4/50
1968/1968 ━━━━━━━━━━━━━━━━━━━━ 693s 352ms/step - accuracy: 0.5700 - loss: 1.3313 - val_accuracy: 0.6089 - val_loss: 1.2474 - learning_rate: 0.0010
Epoch 5/50
1968/1968 ━━━━━━━━━━━━━━━━━━━━ 686s 349ms/step - accuracy: 0.5823 - loss: 1.2586 - val_accuracy: 0.6187 - val_loss: 1.2027 - learning_rate: 0.0010
Epoch 6/50
1968/1968 ━━━━━━━━━━━━━━━━━━━━ 794s 403ms/step - accuracy: 0.5928 - loss: 1.2044 - val_accuracy: 0.5672 - val_loss: 1.4177 - learning_rate: 0.0010
Epoch 7/50
1968/1968 ━━━━━━━━━━━━━━━━━━━━ 694s 352

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import os

# Load the trained model
model = tf.keras.models.load_model("sign_language_model_transfer_alpha.keras")

# Load class indices
class_indices = {v: k for k, v in enumerate(sorted(os.listdir("./Data")))}  # Auto-detect classes
index_to_class = {v: k for k, v in class_indices.items()}

# Initialize MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Preprocess Frame Function
def preprocess_frame(frame):
    tensor = tf.image.resize(frame, [128, 128])  # Resize to 128x128
    tensor = tf.cast(tensor, tf.float32) / 255.0  # Normalize
    tensor = tf.expand_dims(tensor, axis=0)      # Add batch dimension
    return tensor

# Initialize Webcam
cap = cv2.VideoCapture(0)

with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Flip frame for natural webcam view
        frame = cv2.flip(frame, 1)

        # Convert to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        # Draw Hand Landmarks & Predict Sign
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Extract ROI for prediction (Use the whole frame or adjust as needed)
            combined_hand_img = rgb_frame

            # Preprocess for model
            combined_hand_img = preprocess_frame(combined_hand_img)

            # Make Prediction
            predictions = model.predict(combined_hand_img)
            pred_index = np.argmax(predictions[0])
            pred_class = index_to_class.get(pred_index, "Unknown")
            confidence = np.max(predictions[0]) * 100

            # Display Prediction
            cv2.putText(frame, f'{pred_class} ({confidence:.2f}%)', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        else:
            cv2.putText(frame, "No Hands Detected", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Show Frame
        cv2.imshow('SignLoom_Alpha&Digits', frame)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release Resources
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━